In [4]:
# Convex Optimization
# Tutorial 9
# Power assignment in a wireless communication system

import numpy as np
import cvxpy as cp


In [5]:

n = 5

G = np.matrix([[1.0, 0.1, 0.2, 0.1, 0.0],
               [0.1, 1.0, 0.1, 0.1, 0.0],
               [0.2 ,0.1, 2.0, 0.2, 0.2],
               [0.1, 0.1, 0.2, 1.0, 0.1],
               [0.0, 0.0, 0.2, 0.1, 1.0]])

max_pow_transmitter = 3
max_pow_receiver = 5
total_pow_limit_1 = 4
total_pow_limit_2 = 6
upper = 1e4
lower = 0.0

Noise = np.full((n), 0.5)
unit = np.identity(n)
P = cp.Variable(n)
S = cp.multiply(G, unit)
SINR_ratio = cp.Parameter(1)

I = G - S

Objective = cp.Maximize(SINR_ratio)

while (upper - lower >= 0.05):
  SINR_ratio.value = np.atleast_1d((upper + lower)/2.0)
  Constraints = [P >= 0, P <= max_pow_transmitter,
                 P[0] + P[1] <= total_pow_limit_1, 
                 P[2] + P[3] + P[4] <= total_pow_limit_2, 
                 G @ P <= max_pow_receiver, 
                 I @ P + Noise <= SINR_ratio @ S @ P]
  prob = cp.Problem(Objective, Constraints)
  prob.solve()

  if prob.status == 'optimal': upper = (upper + lower) / 2.0; answer = P.value
  else: lower = (upper + lower) / 2.0

answer = np.around(answer, 4)


In [6]:
print(f'The maximum of the minimum SINR ratio is {round(1/SINR_ratio.value[0], 4)}')
print(f'The respective transmitter powers for the above SINR ratio are: ')
print(*answer, sep=', ')


The maximum of the minimum SINR ratio is 1.7476
The respective transmitter powers for the above SINR ratio are: 
2.0533, 1.8569, 1.5972, 2.3231, 1.8557
